### Links To CDC for Raw Data

## Covid-19 Infection Data
### url-1: https://data.cdc.gov/Case-Surveillance/United-States-COVID-19-Cases-and-Deaths-by-State-o/9mfq-cb36

## Covid-19 Vaccine Distribution
### url-2: https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-Jurisdi/unsk-b7fc

In [1]:
# import dependencies
import pandas as pd
import datetime as dt
import sqlite3
from sqlite3 import Error
from sqlalchemy import create_engine, inspect, func
import psycopg2
import requests
from sodapy import Socrata

# import cdc key for API
from cdc_token import my_token

In [79]:
# # set a client with a token
# client = Socrata("data.cdc.gov", my_token)

# # results returned as JSON from API / converted to Python list of
# # dictionaries by sodapy.
# results = client.get("unsk-b7fc", limit=50000)

# # Convert to pandas DataFrame and display
# covid_vac_df = pd.DataFrame.from_records(results)
# covid_vac_df

## USA Monthly Cases by State 

In [53]:
# set a client with a token
client = Socrata("data.cdc.gov", my_token)

# results returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("9mfq-cb36", limit=50000)

# Convert to pandas DataFrame and display
covid_df = pd.DataFrame.from_records(results)
covid_df.tail()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,new_death,pnew_death,created_at,consent_cases,consent_deaths,conf_death,prob_death
43375,2020-06-07T00:00:00.000,SD,5438,NaN,NaN,71.0,0,65,0.0,1,2020-06-08T14:55:08.000,N/A,Agree,64.0,1
43376,2021-12-28T00:00:00.000,NY,1833465,NaN,NaN,13006.0,0,23446,42.0,0,2021-12-29T14:30:55.917,Not agree,Not agree,NaN,NaN
43377,2021-09-25T00:00:00.000,RMI,4,4.0,0,0.0,0,0,0.0,0,2021-09-27T13:21:43.831,Agree,Agree,0.0,0
43378,2021-03-10T00:00:00.000,SD,113962,NaN,NaN,209.0,37,1904,3.0,0,2021-03-11T15:36:21.295,N/A,Agree,1546.0,358
43379,2021-05-27T00:00:00.000,OH,1100312,921199.0,179113,732.0,196,19753,0.0,0,2021-05-28T13:29:21.497,Agree,Agree,19753.0,0


In [54]:
# add date columns which only has year and month for use with groupby later on.. 
covid_df['date'] = pd.to_datetime(covid_df['submission_date']).dt.to_period('M').astype(str)
covid_df.tail()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,new_death,pnew_death,created_at,consent_cases,consent_deaths,conf_death,prob_death,date
43375,2020-06-07T00:00:00.000,SD,5438,NaN,NaN,71.0,0,65,0.0,1,2020-06-08T14:55:08.000,N/A,Agree,64.0,1,2020-06
43376,2021-12-28T00:00:00.000,NY,1833465,NaN,NaN,13006.0,0,23446,42.0,0,2021-12-29T14:30:55.917,Not agree,Not agree,NaN,NaN,2021-12
43377,2021-09-25T00:00:00.000,RMI,4,4.0,0,0.0,0,0,0.0,0,2021-09-27T13:21:43.831,Agree,Agree,0.0,0,2021-09
43378,2021-03-10T00:00:00.000,SD,113962,NaN,NaN,209.0,37,1904,3.0,0,2021-03-11T15:36:21.295,N/A,Agree,1546.0,358,2021-03
43379,2021-05-27T00:00:00.000,OH,1100312,921199.0,179113,732.0,196,19753,0.0,0,2021-05-28T13:29:21.497,Agree,Agree,19753.0,0,2021-05


In [55]:
covid_df.dtypes

submission_date    object
state              object
tot_cases          object
conf_cases         object
prob_cases         object
new_case           object
pnew_case          object
tot_death          object
new_death          object
pnew_death         object
created_at         object
consent_cases      object
consent_deaths     object
conf_death         object
prob_death         object
date               object
dtype: object

In [56]:
# convert objects to float
covid_df["tot_cases"] =  covid_df.tot_cases.astype(float)
covid_df["tot_death"] = covid_df.tot_death.astype(float)

# select only the columns we gonna use and display df
covid_df = covid_df[["date", "state", "tot_cases", "tot_death"]]
covid_df.tail()

,date,state,tot_cases,tot_death
43375,2020-06,SD,5438.0,65.0
43376,2021-12,NY,1833465.0,23446.0
43377,2021-09,RMI,4.0,0.0
43378,2021-03,SD,113962.0,1904.0
43379,2021-05,OH,1100312.0,19753.0


# group by states to be used later
state_df = covid_df.groupby(['date', 'state']).sum().reset_index()
state_df

# group by states to be used later
state_df = covid_df.groupby('state').sum().reset_index()
state_df

In [57]:
# group by states to be used later
state_df = covid_df.groupby('state').mean().reset_index()
state_df.head()

,state,tot_cases,tot_death
0,AK,49455.966805,279.77455
1,AL,376703.973721,7725.75657
2,AR,236603.858921,3837.00000
3,AS,1.622407,0.00000
4,AZ,575980.094053,11007.35823


In [58]:
# add total recovery columns to df
covid_df["Total_recovered"] = covid_df.tot_cases - covid_df.tot_death
covid_df.tail()

,date,state,tot_cases,tot_death,Total_recovered
43375,2020-06,SD,5438.0,65.0,5373.0
43376,2021-12,NY,1833465.0,23446.0,1810019.0
43377,2021-09,RMI,4.0,0.0,4.0
43378,2021-03,SD,113962.0,1904.0,112058.0
43379,2021-05,OH,1100312.0,19753.0,1080559.0


In [59]:
# groupby date and state and take the mean value to get monthly average..
covid_monthly_df = round(covid_df.groupby(["date", "state"]).mean().reset_index(), 2)
covid_monthly_df

,date,state,tot_cases,tot_death,Total_recovered
0,2020-01,AK,0.00,0.00,0.00
1,2020-01,AL,0.00,0.00,0.00
2,2020-01,AR,0.00,0.00,0.00
3,2020-01,AS,0.00,0.00,0.00
4,2020-01,AZ,0.60,0.00,0.60
...,...,...,...,...,...
1495,2022-01,VT,68605.54,460.23,68145.31
1496,2022-01,WA,929222.00,9974.08,919247.92
1497,2022-01,WI,1182867.85,11371.54,1171496.31
1498,2022-01,WV,350151.31,5407.46,344743.85


In [60]:
# rename columns. name gotta be compatable wth sqlite database naming potocol.
covid_monthly_df = covid_monthly_df.rename(columns={"date": "Date", "state": "State", "tot_cases": "Total_cases", "tot_death": "Total_death"})
covid_monthly_df.head()

,Date,State,Total_cases,Total_death,Total_recovered
0,2020-01,AK,0.0,0.0,0.0
1,2020-01,AL,0.0,0.0,0.0
2,2020-01,AR,0.0,0.0,0.0
3,2020-01,AS,0.0,0.0,0.0
4,2020-01,AZ,0.6,0.0,0.6


In [61]:
# add death and recovery percentage columns. change NA values to zero
covid_monthly_df['Death_percent'] = round((covid_monthly_df.Total_death/covid_monthly_df.Total_cases)*100, 2)
covid_monthly_df['Recovery_percent'] = round((covid_monthly_df.Total_recovered/covid_monthly_df.Total_cases)*100, 2)
covid_monthly_df=covid_monthly_df.fillna(0)
covid_monthly_df.head()

,Date,State,Total_cases,Total_death,Total_recovered,Death_percent,Recovery_percent
0,2020-01,AK,0.0,0.0,0.0,0.0,0.0
1,2020-01,AL,0.0,0.0,0.0,0.0,0.0
2,2020-01,AR,0.0,0.0,0.0,0.0,0.0
3,2020-01,AS,0.0,0.0,0.0,0.0,0.0
4,2020-01,AZ,0.6,0.0,0.6,0.0,100.0


In [62]:
# save df as a csv file
covid_monthly_df.to_csv('../Data/monthly.csv')

## USA Covid-19 Vaccine Status by States

In [63]:
# set a client with a token
client = Socrata("data.cdc.gov", my_token)

# results returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("unsk-b7fc", limit=50000)

# Convert to pandas DataFrame and display
covid_vac_df = pd.DataFrame.from_records(results)
covid_vac_df.head()

,date,mmwr_week,location,distributed,distributed_janssen,distributed_moderna,distributed_pfizer,distributed_unk_manuf,dist_per_100k,distributed_per_100k_12plus,...,administered_dose1_recip_5pluspop_pct,series_complete_5plus,series_complete_5pluspop_pct,administered_5plus,admin_per_100k_5plus,distributed_per_100k_5plus,series_complete_moderna_5plus,series_complete_pfizer_5plus,series_complete_janssen_5plus,series_complete_unk_manuf_5plus
0,2022-01-14T00:00:00.000,2,AR,5383350,248700,2214760,2919890,0,178386,210513,...,68,1564912,55.3,3911046,138232,190269,643245,811229,109296,1142
1,2022-01-14T00:00:00.000,2,MT,1801355,103700,734960,962695,0,168543,196312,...,66.5,582451,57.8,1472339,146120,178773,230749,298471,52799,432
2,2022-01-14T00:00:00.000,2,IN,11675870,577200,4087200,7011470,0,173433,204659,...,62.7,3536892,56,8764025,138806,184924,1223080,2031991,273847,7974
3,2022-01-14T00:00:00.000,2,GU,291260,22500,79580,189180,0,172866,221801,...,93.4,128615,84.2,300515,196722,190664,38495,79494,10616,10
4,2022-01-14T00:00:00.000,2,MN,11356950,494200,3980660,6882090,0,201378,237688,...,77.2,3736354,70.7,9424010,178215,214768,1270958,2142882,321397,1117


In [64]:
covid_vac_df.dtypes

date                               object
mmwr_week                          object
location                           object
distributed                        object
distributed_janssen                object
                                    ...  
distributed_per_100k_5plus         object
series_complete_moderna_5plus      object
series_complete_pfizer_5plus       object
series_complete_janssen_5plus      object
series_complete_unk_manuf_5plus    object
Length: 80, dtype: object

In [65]:
# add date columns which only has year and month for use with groupby later on.. 
covid_vac_df['Date'] = pd.to_datetime(covid_vac_df['date']).dt.to_period('M').astype(str)
covid_vac_df.tail()

,date,mmwr_week,location,distributed,distributed_janssen,distributed_moderna,distributed_pfizer,distributed_unk_manuf,dist_per_100k,distributed_per_100k_12plus,...,series_complete_5plus,series_complete_5pluspop_pct,administered_5plus,admin_per_100k_5plus,distributed_per_100k_5plus,series_complete_moderna_5plus,series_complete_pfizer_5plus,series_complete_janssen_5plus,series_complete_unk_manuf_5plus,Date
25683,2020-12-13T00:00:00.000,51,US,13650,0,0,0,0,4,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,2020-12
25684,2020-12-13T00:00:00.000,51,GU,3900,0,0,0,0,2353,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,2020-12
25685,2020-12-13T00:00:00.000,51,AS,3900,0,0,0,0,7003,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,2020-12
25686,2020-12-13T00:00:00.000,51,MP,4875,0,0,0,0,8570,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,2020-12
25687,2020-12-13T00:00:00.000,51,VI,975,0,0,0,0,931,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,2020-12


In [66]:
# convert objects to float
covid_vac_df["distributed"] =  covid_vac_df.distributed.astype(float)
covid_vac_df["dist_per_100k"] = covid_vac_df.dist_per_100k.astype(float)

# select only the columns we gonna use and display df
covid_vac_df = covid_vac_df[["Date", "location", "distributed", "dist_per_100k"]]
covid_vac_df.head()

,Date,location,distributed,dist_per_100k
0,2022-01,AR,5383350.0,178386.0
1,2022-01,MT,1801355.0,168543.0
2,2022-01,IN,11675870.0,173433.0
3,2022-01,GU,291260.0,172866.0
4,2022-01,MN,11356950.0,201378.0


In [67]:
# rename columns. name gotta be compatable wth sqlite database naming potocol.
covid_vac_df = covid_vac_df.rename(columns={"location": "State", "distributed": "Distributed", "dist_per_100k": "Dist_per_100k"})
covid_vac_df.tail()

,Date,State,Distributed,Dist_per_100k
25683,2020-12,US,13650.0,4.0
25684,2020-12,GU,3900.0,2353.0
25685,2020-12,AS,3900.0,7003.0
25686,2020-12,MP,4875.0,8570.0
25687,2020-12,VI,975.0,931.0


In [68]:
covid_vac_df = covid_vac_df.drop_duplicates(subset=['State'], keep='first', inplace=False, ignore_index=True)
covid_vac_df.head(63)

,Date,State,Distributed,Dist_per_100k
0,2022-01,AR,5383350.0,178386.0
1,2022-01,MT,1801355.0,168543.0
2,2022-01,IN,11675870.0,173433.0
3,2022-01,GU,291260.0,172866.0
4,2022-01,MN,11356950.0,201378.0
...,...,...,...,...
58,2022-01,ME,3038950.0,226077.0
59,2022-01,WY,889865.0,153754.0
60,2022-01,VT,1484680.0,237934.0
61,2022-01,IH2,2649435.0,127205.0


In [69]:
covid_vac_df.dtypes

Date              object
State             object
Distributed      float64
Dist_per_100k    float64
dtype: object

In [50]:
# covid_vac_df = covid_vac_df.groupby(['Date', 'State']).mean().reset_index()
# covid_vac_df.head(70)

In [51]:
# covid_test = covid_vac_df.loc[(covid_vac_df['State'] == 'TX')]
# covid_test

In [75]:
# covid_vac_df = covid_vac_df.groupby('Location').sum().reset_index()
# covid_vac_df.tail(50)

In [70]:
# save df as a csv file
covid_vac_df.to_csv('../Data/vaccine.csv')

## USA Total Cases and Death by State

In [71]:
# upload data file and read it ito a pandas dataframe
file = "../Data/US_COVID-19_Deaths.csv"
covid_stats_df = pd.read_csv(file)
covid_stats_df.head()

,State/Territory,Total Cases,Confirmed Cases,Probable Cases,Cases in Last 7 Days,Case Rate per 100000,Total Deaths,Confirmed Deaths,Probable Deaths,Deaths in Last 7 Days,Death Rate per 100000,Case Rate per 100000 in Last 7 Days,Death Rate per 100K in Last 7 Days
0,Alaska,17072,NaN,NaN,2616,2315,84,NaN,NaN,7,11,50.7,0.1
1,Alabama,199158,169266.0,29892.0,10009,4074,3026,2818.0,208.0,112,61,29.3,0.3
2,Arkansas,117360,NaN,NaN,7648,3894,2037,NaN,NaN,143,67,36.3,0.7
3,American Samoa,0,NaN,NaN,0,0,0,NaN,NaN,0,0,0.0,0.0
4,Arizona,252768,246468.0,6300.0,10288,3524,6087,5707.0,380.0,169,84,20.5,0.3


In [72]:
# select relevant columns only
covid_stats_df = covid_stats_df[['State/Territory', 'Total Cases', 'Case Rate per 100000', 'Total Deaths', 'Death Rate per 100000']]
covid_stats_df.head()

,State/Territory,Total Cases,Case Rate per 100000,Total Deaths,Death Rate per 100000
0,Alaska,17072,2315,84,11
1,Alabama,199158,4074,3026,61
2,Arkansas,117360,3894,2037,67
3,American Samoa,0,0,0,0
4,Arizona,252768,3524,6087,84


In [73]:
# add a state abbreviation column
covid_stats_df['State'] = state_df['state']
covid_stats_df.head()

,State/Territory,Total Cases,Case Rate per 100000,Total Deaths,Death Rate per 100000,State
0,Alaska,17072,2315,84,11,AK
1,Alabama,199158,4074,3026,61,AL
2,Arkansas,117360,3894,2037,67,AR
3,American Samoa,0,0,0,0,AS
4,Arizona,252768,3524,6087,84,AZ


In [74]:
# rename columns and drop any column with NA
covid_stats_df = covid_stats_df.rename(columns={"state": "State", "Total Cases": "Total_cases", "Case Rate per 100000": "Case_rate_per_100k", "Total Deaths": "Total_deaths", "Death Rate per 100000": "Death_rate_per_100k"})
covid_stats_df = covid_stats_df[["State", "Total_cases", "Case_rate_per_100k", "Total_deaths", "Death_rate_per_100k"]].fillna(0)
covid_stats_df.tail()

,State,Total_cases,Case_rate_per_100k,Total_deaths,Death_rate_per_100k
56,WA,112550,1494,2431,32
57,WI,263571,4534,2269,39
58,WV,26547,1470,480,26
59,WY,15409,2667,105,18
60,0,9581770,2895,234264,71


## USA COVID-19 data with latitude and longtitude to create interactive map

In [75]:
# upload file and read to pandas dataframe
file = "../Data/US_COVID-19_stats.csv"
us_covid_df = pd.read_csv(file)
us_covid_df.head()

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
0,Alabama,US,2020-11-06 05:30:35,32.3182,-86.9023,199158,3026,84471.0,111661.0,1.0,4061.808804,1379107.0,NaN,1.519397,84000001,USA,28126.758423,NaN
1,Alaska,US,2020-11-06 05:30:35,61.3707,-152.4044,18174,84,7125.0,10965.0,2.0,2484.331107,781826.0,NaN,0.462199,84000002,USA,106873.261385,NaN
2,American Samoa,US,2020-11-06 05:30:35,-14.2710,-170.1320,0,0,NaN,0.0,60.0,0.000000,1768.0,NaN,NaN,16,ASM,3177.512985,NaN
3,Arizona,US,2020-11-06 05:30:35,33.7298,-111.4312,252768,6087,42317.0,204364.0,4.0,3472.699928,1826629.0,NaN,2.408137,84000004,USA,25095.480426,NaN
4,Arkansas,US,2020-11-06 05:30:35,34.9697,-92.3731,117360,2037,104816.0,10507.0,5.0,3888.920553,1392905.0,NaN,1.735685,84000005,USA,46156.244740,NaN


In [76]:
# select relevant columns
us_covid_df = us_covid_df[["Province_State", "Lat", "Long_", "Confirmed", "Deaths", "Recovered", "Mortality_Rate"]]
us_covid_df.head()

,Province_State,Lat,Long_,Confirmed,Deaths,Recovered,Mortality_Rate
0,Alabama,32.3182,-86.9023,199158,3026,84471.0,1.519397
1,Alaska,61.3707,-152.4044,18174,84,7125.0,0.462199
2,American Samoa,-14.2710,-170.1320,0,0,NaN,NaN
3,Arizona,33.7298,-111.4312,252768,6087,42317.0,2.408137
4,Arkansas,34.9697,-92.3731,117360,2037,104816.0,1.735685


In [77]:
# add state column to be filled with state abbreviation
us_covid_df["State"] = "" 
us_covid_df.head()

,Province_State,Lat,Long_,Confirmed,Deaths,Recovered,Mortality_Rate,State
0,Alabama,32.3182,-86.9023,199158,3026,84471.0,1.519397,
1,Alaska,61.3707,-152.4044,18174,84,7125.0,0.462199,
2,American Samoa,-14.2710,-170.1320,0,0,NaN,NaN,
3,Arizona,33.7298,-111.4312,252768,6087,42317.0,2.408137,
4,Arkansas,34.9697,-92.3731,117360,2037,104816.0,1.735685,


In [78]:
# upload States and their abbreviation
us_state_abbrev = {
    
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Diamond Princess': 'DP',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Grand Princess': 'GP',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

In [79]:
# loop through and assign abbreviation acoordinely
for index, row in us_covid_df.iterrows():
    us_covid_df.loc[index, "State"] = us_state_abbrev[row[0]]

us_covid_df.head()

,Province_State,Lat,Long_,Confirmed,Deaths,Recovered,Mortality_Rate,State
0,Alabama,32.3182,-86.9023,199158,3026,84471.0,1.519397,AL
1,Alaska,61.3707,-152.4044,18174,84,7125.0,0.462199,AK
2,American Samoa,-14.2710,-170.1320,0,0,NaN,NaN,AS
3,Arizona,33.7298,-111.4312,252768,6087,42317.0,2.408137,AZ
4,Arkansas,34.9697,-92.3731,117360,2037,104816.0,1.735685,AR


In [ ]:
# loop through and assign abbreviation acoordinely
# for index, row in us_covid_df.iterrows():
#     if row["State"] == "DP":
#         us_covid_df.loc[index, "Lat"] = 14.5214
#         us_covid_df.loc[index, "Long_"] = 120.9709

#     elif row["State"] == "GP":
#         us_covid_df.loc[index, "Lat"] = 32.1584
#         us_covid_df.loc[index, "Long_"] = 117.5676

    # if row["State"] == FSM:
    #     us_covid_df.loc[index, "Lat"] = 6.8874
    #     us_covid_df.loc[index, "Long"] = 158.2150

    # elif row["State"] == NYC:
    #     us_covid_df.loc[index, "Lat"] = 40.7128
    #     us_covid_df.loc[index, "Long"] = -74.0060

    # elif row["State"] == PW:
    #     us_covid_df.loc[index, "Lat"] = 7.5150
    #     us_covid_df.loc[index, "Long"] = 134.5825

    # elif row["State"] == RMI:
    #     us_covid_df.loc[index, "Lat"] = 7.1315
    #     us_covid_df.loc[index, "Long"] = 171.1845

# us_covid_df.head()

In [80]:
# rename columns and add recovery percent column
us_covid_df = us_covid_df.rename(columns={"Long_": "Long", "Mortality_Rate":"Death_percent"})
us_covid_df["Recovery_percent"]=us_covid_df.Recovered/us_covid_df.Confirmed
us_covid_df = us_covid_df[["State", "Lat", "Long", "Death_percent", "Recovery_percent"]].fillna(0)
us_covid_df.head()

,State,Lat,Long,Death_percent,Recovery_percent
0,AL,32.3182,-86.9023,1.519397,0.424141
1,AK,61.3707,-152.4044,0.462199,0.392044
2,AS,-14.2710,-170.1320,0.000000,0.000000
3,AZ,33.7298,-111.4312,2.408137,0.167414
4,AR,34.9697,-92.3731,1.735685,0.893115


In [81]:
# loop through and assign abbreviation acoordinely
for index, row in us_covid_df.iterrows():
    if row["State"] == "DP":
        us_covid_df.loc[index, "Lat"] = 14.5214
        us_covid_df.loc[index, "Long"] = 120.9709

    elif row["State"] == "GP":
        us_covid_df.loc[index, "Lat"] = 32.1584
        us_covid_df.loc[index, "Long"] = 117.5676

    # elif row["State"] == FSM:
    #     us_covid_df.loc[index, "Lat"] = 6.8874
    #     us_covid_df.loc[index, "Long"] = 158.2150

    # elif row["State"] == NYC:
    #     us_covid_df.loc[index, "Lat"] = 40.7128
    #     us_covid_df.loc[index, "Long"] = -74.0060

    # elif row["State"] == PW:
    #     us_covid_df.loc[index, "Lat"] = 7.5150
    #     us_covid_df.loc[index, "Long"] = 134.5825

    # elif row["State"] == RMI:
    #     us_covid_df.loc[index, "Lat"] = 7.1315
    #     us_covid_df.loc[index, "Long"] = 171.1845

us_covid_df.head()

,State,Lat,Long,Death_percent,Recovery_percent
0,AL,32.3182,-86.9023,1.519397,0.424141
1,AK,61.3707,-152.4044,0.462199,0.392044
2,AS,-14.2710,-170.1320,0.000000,0.000000
3,AZ,33.7298,-111.4312,2.408137,0.167414
4,AR,34.9697,-92.3731,1.735685,0.893115


In [82]:
# merge two df into one dataframe using left joint.
us_covid_combine_df = pd.merge(covid_stats_df, us_covid_df, how='left', on=['State','State'])
us_covid_combine_df.head()

,State,Total_cases,Case_rate_per_100k,Total_deaths,Death_rate_per_100k,Lat,Long,Death_percent,Recovery_percent
0,AK,17072,2315,84,11,61.3707,-152.4044,0.462199,0.392044
1,AL,199158,4074,3026,61,32.3182,-86.9023,1.519397,0.424141
2,AR,117360,3894,2037,67,34.9697,-92.3731,1.735685,0.893115
3,AS,0,0,0,0,-14.2710,-170.1320,0.000000,0.000000
4,AZ,252768,3524,6087,84,33.7298,-111.4312,2.408137,0.167414


In [83]:
# save to a csv file
us_covid_combine_df.to_csv('../Data/states.csv')

## Add Geo property to states in old state file

In [84]:
# Add Lat and Long Columns
geomap_df = covid_monthly_df.groupby('State').mean().reset_index()
# covid_monthly_df["Long"] = ""
geomap_df.head()

,State,Total_cases,Total_death,Total_recovered,Death_percent,Recovery_percent
0,AK,50545.5468,287.0944,50258.4528,1.0088,90.9912
1,AL,379679.1412,7715.5356,371963.6048,inf,-inf
2,AR,238437.2540,3855.5256,234581.7296,1.4532,90.5468
3,AS,1.7924,0.0000,1.7924,0.0000,40.0000
4,AZ,580049.1420,11013.9744,569035.1672,2.0288,97.9712


## World COVID-19 Data by Country

In [85]:
# upload file and read into a pandas dataframe
file = "../Data/World_COVID-19_Stats.csv"
world_covid_df = pd.read_csv(file)
world_covid_df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-11-06 05:24:55,33.93911,67.709953,41935,1554,34440,5941.0,Afghanistan,107.723573,3.705735
1,NaN,NaN,NaN,Albania,2020-11-06 05:24:55,41.15330,20.168300,22721,543,11696,10482.0,Albania,789.526722,2.389860
2,NaN,NaN,NaN,Algeria,2020-11-06 05:24:55,28.03390,1.659600,60169,2011,41244,16914.0,Algeria,137.212244,3.342253
3,NaN,NaN,NaN,Andorra,2020-11-06 05:24:55,42.50630,1.521800,5135,75,3858,1202.0,Andorra,6645.958714,1.460565
4,NaN,NaN,NaN,Angola,2020-11-06 05:24:55,-11.20270,17.873900,12102,299,5350,6453.0,Angola,36.821948,2.470666


In [86]:
# select relevant columns only
world_covid_df = world_covid_df[["Country_Region", "Lat", "Long_", "Confirmed", "Deaths", "Recovered", "Case-Fatality_Ratio"]]
world_covid_df.head()

,Country_Region,Lat,Long_,Confirmed,Deaths,Recovered,Case-Fatality_Ratio
0,Afghanistan,33.93911,67.709953,41935,1554,34440,3.705735
1,Albania,41.15330,20.168300,22721,543,11696,2.389860
2,Algeria,28.03390,1.659600,60169,2011,41244,3.342253
3,Andorra,42.50630,1.521800,5135,75,3858,1.460565
4,Angola,-11.20270,17.873900,12102,299,5350,2.470666


In [87]:
# rename columns and add a recovery percent column
world_covid_df = world_covid_df.rename(columns={"Long_": "Long", "Case-Fatality_Ratio":"Death_percent", "Country_Region": "Country"})
world_covid_df["Recovery_percent"]=world_covid_df.Recovered/world_covid_df.Confirmed
world_covid_df.head()

,Country,Lat,Long,Confirmed,Deaths,Recovered,Death_percent,Recovery_percent
0,Afghanistan,33.93911,67.709953,41935,1554,34440,3.705735,0.821271
1,Albania,41.15330,20.168300,22721,543,11696,2.389860,0.514766
2,Algeria,28.03390,1.659600,60169,2011,41244,3.342253,0.685469
3,Andorra,42.50630,1.521800,5135,75,3858,1.460565,0.751315
4,Angola,-11.20270,17.873900,12102,299,5350,2.470666,0.442076


In [88]:
# add missing geoinfo to country
for index, row in world_covid_df.iterrows(): 
    if row["Country"] == "Canada":
        world_covid_df.loc[index, "Lat"] = 56.1304
        world_covid_df.loc[index, "Long"] = 106.3468
world_covid_df

,Country,Lat,Long,Confirmed,Deaths,Recovered,Death_percent,Recovery_percent
0,Afghanistan,33.939110,67.709953,41935,1554,34440,3.705735,0.821271
1,Albania,41.153300,20.168300,22721,543,11696,2.389860,0.514766
2,Algeria,28.033900,1.659600,60169,2011,41244,3.342253,0.685469
3,Andorra,42.506300,1.521800,5135,75,3858,1.460565,0.751315
4,Angola,-11.202700,17.873900,12102,299,5350,2.470666,0.442076
...,...,...,...,...,...,...,...,...
3955,West Bank and Gaza,31.952200,35.233200,56672,508,48680,0.896386,0.858978
3956,Western Sahara,24.215500,-12.885800,10,1,8,10.000000,0.800000
3957,Yemen,15.552727,48.516388,2063,601,1375,29.132332,0.666505
3958,Zambia,-13.133897,27.849332,16770,349,15827,2.081097,0.943769


In [89]:
# save file as a csv
world_covid_df.to_csv('../Data/world.csv')

## build SQlite DataBase

In [90]:
# import modules
import sqlite3
from sqlite3 import Error

# series of functions to  build database tables in sqlite database.

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return conn
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def drop_table(conn, drop_table_sql):
    """ drop a table from the drop_table_sql statement
    :param conn: Connection object
    :param drop_table_sql: a drop TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(drop_table_sql)
    except Error as e:
        print(e)

# build empty tables with correct columns names
def main():		
    database = r"../DataBase/covid-19.db"
    sql_create_monthly_table = """CREATE TABLE monthly(
                                        id INTEGER PRIMARY KEY AUTOINCREMENT, 
                                        Date text NOT NULL,
                                        State text,
                                        Total_cases float,
                                        Total_death float,
                                        Total_recovered float,
                                        Death_percent float,
                                        Recovery_percent float
                                    ); """

    sql_drop_monthly_table = """DROP TABLE if exists monthly;"""

    sql_create_vaccine_table = """CREATE TABLE vaccine(
                                        id INTEGER PRIMARY KEY AUTOINCREMENT, 
                                        Date text NOT NULL,
                                        State text,
                                        Distributed float,
                                        Dist_per_100k float
                                    ); """

    sql_drop_vaccine_table = """DROP TABLE if exists vaccine;"""
    
    sql_create_states_table = """CREATE TABLE states(
                                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                                        State text,
                                        Total_cases float,
                                        Case_rate_per_100k float,
                                        Total_deaths float,
                                        Death_rate_per_100k float,
                                        Lat float,
                                        Long float,
                                        Death_percent float,
                                        Recovery_percent float
                                    );"""

    sql_drop_states_table = """DROP TABLE if exists states;"""    

    sql_create_world_table = """CREATE TABLE world(
                                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                                        Country text,
                                        Lat float,
                                        Long float,
                                        Confirmed float,
                                        Deaths float,
                                        Recovered float,
                                        Recovery_percent float,
                                        Death_percent float
                                    );"""

    sql_drop_world_table = """DROP TABLE if exists world;"""

    # create a database connection
    conn = create_connection(database)
    
    # create tables
    if conn is not None:
        # create covid_monthly table
        drop_table(conn, sql_drop_monthly_table)
        create_table(conn, sql_create_monthly_table)

        # create covid_vaccine table
        drop_table(conn, sql_drop_vaccine_table)
        create_table(conn, sql_create_vaccine_table)

        # create covid_states table
        drop_table(conn, sql_drop_states_table)
        create_table(conn, sql_create_states_table)

        #create covid_world table
        drop_table(conn, sql_drop_world_table)
        create_table(conn, sql_create_world_table)
        
    # error handling
    else:
        print("Error! cannot create the database connection.")
if __name__ == '__main__':
    main()

In [91]:
# create engine connection to database
engine = create_engine('sqlite:///../DataBase/covid-19.db')

# # populate tables with information from dataframes
covid_monthly_df.to_sql(name='monthly', con=engine, if_exists = "append", index=False)
covid_vac_df.to_sql(name='vaccine', con=engine, if_exists = "append", index=False)
world_covid_df.to_sql(name='world', con=engine, if_exists = "append", index= False)
us_covid_combine_df.to_sql(name='states', con=engine, if_exists = "append", index=False)

In [92]:
# querry monthly data from database. to test and make sure tables are working
pd.read_sql('select * from monthly', engine)

,id,Date,State,Total_cases,Total_death,Total_recovered,Death_percent,Recovery_percent
0,1,2020-01,AK,0.00,0.00,0.00,0.00,0.00
1,2,2020-01,AL,0.00,0.00,0.00,0.00,0.00
2,3,2020-01,AR,0.00,0.00,0.00,0.00,0.00
3,4,2020-01,AS,0.00,0.00,0.00,0.00,0.00
4,5,2020-01,AZ,0.60,0.00,0.60,0.00,100.00
...,...,...,...,...,...,...,...,...
1495,1496,2022-01,VT,68605.54,460.23,68145.31,0.67,99.33
1496,1497,2022-01,WA,929222.00,9974.08,919247.92,1.07,98.93
1497,1498,2022-01,WI,1182867.85,11371.54,1171496.31,0.96,99.04
1498,1499,2022-01,WV,350151.31,5407.46,344743.85,1.54,98.46


In [93]:
pd.read_sql('select * from vaccine', engine)

,id,Date,State,Distributed,Dist_per_100k
0,1,2022-01,AR,5383350.0,178386.0
1,2,2022-01,MT,1801355.0,168543.0
2,3,2022-01,IN,11675870.0,173433.0
3,4,2022-01,GU,291260.0,172866.0
4,5,2022-01,MN,11356950.0,201378.0
...,...,...,...,...,...
60,61,2022-01,VT,1484680.0,237934.0
61,62,2022-01,IH2,2649435.0,127205.0
62,63,2022-01,UT,5634810.0,175761.0
63,64,2022-01,MS,4934115.0,165789.0


In [94]:
# querry states data from database. to test and make sure tables are working
pd.read_sql('select * from states', engine)

,id,State,Total_cases,Case_rate_per_100k,Total_deaths,Death_rate_per_100k,Lat,Long,Death_percent,Recovery_percent
0,1,AK,17072.0,2315.0,84.0,11.0,61.3707,-152.4044,0.462199,0.392044
1,2,AL,199158.0,4074.0,3026.0,61.0,32.3182,-86.9023,1.519397,0.424141
2,3,AR,117360.0,3894.0,2037.0,67.0,34.9697,-92.3731,1.735685,0.893115
3,4,AS,0.0,0.0,0.0,0.0,-14.2710,-170.1320,0.000000,0.000000
4,5,AZ,252768.0,3524.0,6087.0,84.0,33.7298,-111.4312,2.408137,0.167414
...,...,...,...,...,...,...,...,...,...,...
56,57,WA,112550.0,1494.0,2431.0,32.0,47.4009,-121.4905,2.159929,0.000000
57,58,WI,263571.0,4534.0,2269.0,39.0,44.2685,-89.6165,0.877867,0.773711
58,59,WV,26547.0,1470.0,480.0,26.0,38.4912,-80.9545,1.815648,0.759973
59,60,WY,15409.0,2667.0,105.0,18.0,42.7560,-107.3025,0.681420,0.630086


In [ ]:
# make an API call to make sure Flask app is working 
import requests
url="http://127.0.0.1:5000/api/v1.0/monthly"
resp=requests.get(url)
resp

In [ ]:
data = resp.json()
data